# Model - Object Captioning

Here I will run a model to generate captions for objects. This will be done in a few step.  
(the main structure of steps and dataset from https://machinelearningmastery.com/develop-a-deep-learning-caption-generation-model-in-python/)

1. Extract feature from pretrained network for object recognition.  
2. Using the flicker dataset with labels, train CNN/RNN model 
3. Art Captioning: for this I'll try couple different method
    - test artworks on a model trained for object captioning
    - include art with description to the CNN/RNN 
    - include artworks to the CNN
    - include less concrete object images (e.g. cloud) to the CNN
    - train more abstract labeling in RNN
    
Additionally, some evaluation steps to take a look.
1. Check what type of objects are classified better or worse
2. Try semantic projection (abstract - concrete) to score the level of abstraction of each word and run statistical testing on how the model performance differs per the level of abstraction
3. 

In [34]:
from tensorflow.keras.applications import NASNetLarge 
from tensorflow.keras.applications.nasnet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model

import pickle

import os

import numpy as np

In [9]:
flicker_img_dir = 'IMAGES/Flicker/Flicker8k_Dataset'
flicker_text_dir = 'IMAGES/Flicker/labels'

## Feature Extraction
First, extracting features from Flicker dataset using NASNet network.

In [56]:
def feature_extractor(dir_, network):
    ''' 
    iterate through files in dir_ 
    and get features running on network
    return a dictionary with image id as a key
    '''
    model = network(include_top = False)
    fnames = [x for x in os.listdir(dir_) if x.endswith('.jpg')]
    result = {}
    i = 1
    n = len(fnames)
    
    for fn in fnames:
        img = load_img(f'{dir_}/{fn}', target_size = (model.input.shape[1], model.input.shape[2]))
        img = np.expand_dims(img, 0)
        img = preprocess_input(img)
        feature = model.predict(img)
        ind = fn[:-4]
        result[ind] = feature
        print(f'{i}/{n} feature extraction completed')
        i += 1
    return result

In [ ]:
features = feature_extractor(flicker_img_dir, NASNetLarge)